In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
from pathlib import Path

src_path = Path('/u/trappist-r0/govorcin/01_OPERA/TROPO/opera_tropo/src')
sys.path.append(str(src_path))

In [14]:
from opera_tropo import product_info
from opera_tropo.config import pge_runconfig, runconfig
from opera_tropo.utils import get_hres_datetime

ImportError: cannot import name 'product_info' from 'opera_tropo' (unknown location)

# Product info
metadata of the output product

In [4]:
tropo = product_info.TROPO_PRODUCTS

In [5]:
for item, key in tropo.__dict__.items():
    print(item,'\n',50*'#')
    for i, ik in key.__dict__.items():
        if item == 'coords':
            print(i)
            for ix, ik in ik.__dict__.items():
                print(' ', ix,':', ik)
        else:
            print(i,':', ik)

    print('\n')

wet_delay 
 ##################################################
name : wet_delay
long_name : one way zenith wet delay
description : Total Zenith Wet Delay Correction at the top of the atmosphere
fillvalue : nan
dtype : <class 'numpy.float32'>
attrs : {'units': 'meters', 'grid_mapping': 'spatial_ref'}
keep_bits : 12


hydrostatic_delay 
 ##################################################
name : hydrostatic_delay
long_name : one way zenith hydrostatic delay
description : Total Zenith Wet Delay Correction at the top of the atmosphere
fillvalue : nan
dtype : <class 'numpy.float32'>
attrs : {'units': 'meters', 'grid_mapping': 'spatial_ref'}
keep_bits : 12


coords 
 ##################################################
latitude
  axis : y
  units : degrees
  standard_name : latitude
  long_name : latitude
  description : Angular distance of a point north or south of the equator.
  encoding : None
longitude
  axis : x
  units : degrees
  standard_name : longitude
  long_name : longitude
  descri

# Configuration 

In [13]:
tropo_workflow = runconfig.TropoWorkflow
tropo_workflow = tropo_workflow.from_yaml('opera_tropo/src/opera_tropo/config/default/runconfig.yaml')

In [9]:
# Print config defaults
tropo_workflow.print_yaml_schema()

input_options:
  # Path to the input HRES model hres_model.nc.
  #   Type: string.
  input_file_path: ''
  # Format of dates contained in s3 HRES folder.
  #   Type: string.
  date_fmt: '%Y%m%d'
output_options:
  # Output Date Format for OPERA TROPO.
  #   Type: string.
  date_fmt: '%Y%m%dT%H%M%S'
  # Time the config file was created.
  #   Type: string.
  creation_time: '2025-01-30T02:56:55.630819Z'
  # Output height level to hydrostatic and wet delay default:  RAiDER HRES 145 height levels.
  #   Type: array | null.
  output_heights: []
  # Product output compression options for netcdf.
  #   Type: object | null.
  compression_kwargs:
    compression_flag: false
    zlib: true
    complevel: 4
    shuffle: true
  # OPERA TROPO product version.
  #   Type: string.
  product_version: '0.1'
# Name of directory to use for writing output files.
#   Type: string.
work_directory: /u/trappist-r0/govorcin/01_OPERA/TROPO
# Don't resolve filepaths that are given as relative to be absolute.
#   

In [15]:
# Print configuration parameters
tropo_workflow.model_dump()

{'input_options': {'input_file_path': '/u/trappist-r0/govorcin/01_OPERA/HRES/data/D01010000010100001.subset.zz.nc',
  'date_fmt': '%Y%m%d'},
 'output_options': {'date_fmt': '%Y%m%dT%H%M%S',
  'creation_time': datetime.datetime(2025, 1, 30, 2, 58, 16, 888033, tzinfo=datetime.timezone.utc),
  'output_heights': [],
  'compression_kwargs': {'compression_flag': False,
   'zlib': True,
   'complevel': 5,
   'shuffle': True},
  'product_version': '0.1'},
 'work_directory': PosixPath('/u/trappist-r0/govorcin/01_OPERA/TROPO'),
 'keep_paths_relative': False,
 'worker_settings': {'n_workers': 5,
  'threads_per_worker': 2,
  'max_memory': 8,
  'dask_temp_dir': 'tmp',
  'block_shape': (128, 128)},
 'log_file': PosixPath('/u/trappist-r0/govorcin/01_OPERA/HRES/tropo/log_sas.log')}

In [20]:
# Get model datetime and hour
hres_dt = get_hres_datetime(tropo_workflow.input_options.input_file_path)
hres_dt

('20190101', 0)

In [24]:
# Get output filename
#OPERA_L{level}_{product_name}_{spatial_coverage}_{hres_model_datetime}_{process_start_datetime}_{WEATHER_MODEL}_{HRES_spatial_resolution}_{product_version}.nc
tropo_workflow.output_options.get_output_filename(*hres_dt)

'OPERA_L4_TROPO_GLOBAL_20190101T000000Z_20250130T025816Z_HRES_0.1_v0.1.nc'

In [25]:
#PGE runconfig contains few extra fields
pge_workflow = pge_runconfig.RunConfig()
pge_workflow = pge_workflow.from_yaml('opera_tropo/src/opera_tropo/config/default/pge_runconfig.yaml')
pge_workflow.model_dump()

{'input_file': {'input_file_path': '', 'date_fmt': '%Y%m%d'},
 'output_options': {'date_fmt': '%Y%m%dT%H%M%S',
  'creation_time': datetime.datetime(2025, 1, 30, 3, 6, 22, 296741, tzinfo=datetime.timezone.utc),
  'output_heights': [],
  'compression_kwargs': {'compression_flag': True,
   'zlib': True,
   'complevel': 5,
   'shuffle': True},
  'product_version': '0.1'},
 'primary_executable': {'product_type': 'OPERA_TROPO'},
 'product_path_group': {'product_path': PosixPath('output/historical'),
  'scratch_path': PosixPath('scratch/historical'),
  'output_directory': PosixPath('output/historical'),
  'product_version': '0.1'},
 'worker_settings': {'n_workers': 1,
  'threads_per_worker': 2,
  'max_memory': 4,
  'dask_temp_dir': 'tmp',
  'block_shape': (128, 256)},
 'log_file': PosixPath('scratch/historical/log_sas.log')}

In [29]:
# Get tropowork config from pge_config
pge_workflow.to_workflow(), pge_workflow.to_workflow().model_dump()

(TropoWorkflow(input_options=InputOptions(input_file_path='', date_fmt='%Y%m%d'), output_options=OutputOptions(date_fmt='%Y%m%dT%H%M%S', creation_time=datetime.datetime(2025, 1, 30, 3, 6, 22, 296741, tzinfo=datetime.timezone.utc), output_heights=[], compression_kwargs={'compression_flag': True, 'zlib': True, 'complevel': 5, 'shuffle': True}, product_version='0.1'), work_directory=PosixPath('/u/trappist-r0/govorcin/01_OPERA/TROPO/scratch/historical'), keep_paths_relative=False, worker_settings=WorkerSettings(n_workers=1, threads_per_worker=2, max_memory=4, dask_temp_dir='scratch/historical/tmp', block_shape=(128, 256)), log_file=PosixPath('scratch/historical/log_sas.log')),
 {'input_options': {'input_file_path': '', 'date_fmt': '%Y%m%d'},
  'output_options': {'date_fmt': '%Y%m%dT%H%M%S',
   'creation_time': datetime.datetime(2025, 1, 30, 3, 6, 22, 296741, tzinfo=datetime.timezone.utc),
   'output_heights': [],
   'compression_kwargs': {'compression_flag': True,
    'zlib': True,
    'co